In [6]:
import json
import os
import re
import requests

def filetree(top):
    for dirpath, _, fnames in os.walk(top):
        for fname in fnames:
            json_ = '(-t\d\d)*\.json'
            transcript_ = '-t\d\d\.json'
            yield {
                'filename' : os.path.join(dirpath, fname),
                'case_id'  : re.sub(json_, '', fname),
                'type'     : 'transcript' if re.search(transcript_, fname) else 'decision_data'
            }

case_data = list(filetree('./data'))

In [47]:
with open(case_data[1]['filename'], 'r', encoding='utf-8') as file:
    vote_data = json.loads(file.read())['decisions'][0]
    print(vote_data)

{'href': 'https://api.oyez.org/case_decision/case_decision/16686', 'votes': [{'ideology': 0, 'seniority': 2, 'opinion_type': 'majority', 'member': {'href': 'https://api.oyez.org/people/anthony_m_kennedy', 'ID': 15113, 'last_name': 'Kennedy', 'thumbnail': {'size': 47100, 'href': 'https://api.oyez.org/sites/default/files/images/people/anthony_m_kennedy/anthony_m_kennedy.thumb.png', 'id': 32679, 'mime': 'image/png'}, 'length_of_service': 10275, 'view_count': 158, 'identifier': 'anthony_m_kennedy', 'roles': [{'appointing_president': 'Ronald Reagan', 'href': 'https://api.oyez.org/preson_role/scotus_justice/2757', 'role_title': 'Associate Justice of the Supreme Court of the United States', 'type': 'scotus_justice', 'date_end': 0, 'id': 2757, 'date_start': 572162400, 'institution_name': 'Supreme Court of the United States'}], 'name': 'Anthony M. Kennedy'}, 'href': 'https://api.oyez.org/decision_vote/decision_vote/208102', 'vote': 'majority', 'joining': None}, {'ideology': 1, 'seniority': 1, '

e.g. data <link>https://api.oyez.org/cases/2019/17-1623</link>

In [66]:
class Person:
    def __init__(self, name, is_justice=False, sided_with=None, representing=None) -> None:
        super().__init__()
        self.name = name
        self.is_justice = False
        self.representing = None
        self.sided_with = None
        

class Utterance:
    def __init__(self, speaker:Person, content:str):
        self.speaker = speaker
        self.content = content

class Case:
    def __init__(self, case_name, winner, utterances:list):
        self.casename = case_name
        self.utterances = utterances
        self.winner = winner


votes = []

"""
Assign justice votes to names from summary file.
"""

with open(case_data[1]['filename'], 'r', encoding='utf-8') as file:
    vote_data = json.loads(file.read())
    for vote in vote_data:
        first_party = vote_data['first_party']
        second_party = vote_data['second_party']
        winning_party = vote_data['decisions'][0]['winning_party']
        losing_party = first_party if (first_party != winning_party) else second_party
        votes.append(
            Person(
                name = vote['decisions'][0]['votes'][0]['member']['identifier'],
                is_justice = True,
                sided_with = winning_party if vote['vote'] == 'majority' else losing_party
            )
        )

votes

TypeError: string indices must be integers